In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms

from PIL import Image

In [ ]:
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

normalize_step = transforms.Normalize(
    mean=imagenet_mean,
    std=imagenet_std
)

resnet_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize_step
])

cifar_pipeline = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize_step
])


In [ ]:
class CIFAR10Resized(Dataset):

    def __init__(self,
                 root_dir="cifar-10-batches-py",
                 training=True,
                 transform=None):

        self.transform = transform
        image_chunks = []
        target_list = []

        if training:

            for batch_id in range(1, 6):

                batch_file = os.path.join(
                    root_dir,
                    f"data_batch_{batch_id}"
                )

                with open(batch_file, "rb") as fh:
                    payload = pickle.load(fh, encoding="bytes")

                image_chunks.append(payload[b"data"])
                target_list.extend(payload[b"labels"])

            raw_images = np.vstack(image_chunks).astype(np.uint8)

        else:

            batch_file = os.path.join(root_dir, "test_batch")

            with open(batch_file, "rb") as fh:
                payload = pickle.load(fh, encoding="bytes")

            raw_images = payload[b"data"].astype(np.uint8)
            target_list = payload[b"labels"]

        # convert to HWC layout
        self.images = raw_images.reshape(-1, 3, 32, 32) \
                                 .transpose(0, 2, 3, 1)

        self.targets = target_list

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        img = Image.fromarray(self.images[index])
        label = self.targets[index]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

def build_cifar_resnet_sets(data_root="cifar-10-batches-py"):

    train_pool = CIFAR10Resized(
        root_dir=data_root,
        training=True,
        transform=resnet_pipeline
    )

    split_point = int(0.8 * len(train_pool))

    train_subset, val_subset = random_split(
        train_pool,
        [split_point, len(train_pool) - split_point]
    )

    test_subset = CIFAR10Resized(
        root_dir=data_root,
        training=False,
        transform=resnet_pipeline
    )

    return train_subset, val_subset, test_subset

class CatsDogsResized(Dataset):

    def __init__(self, folder, transform=None):

        self.transform = transform
        self.paths = []
        self.targets = []

        for file_path in glob(os.path.join(folder, "*.jpg")):

            name = os.path.basename(file_path).lower()

            if "cat" in name:
                label = 0
            elif "dog" in name:
                label = 1
            else:
                raise ValueError(f"Invalid label source: {name}")

            self.paths.append(file_path)
            self.targets.append(label)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        img = Image.open(self.paths[index]).convert("RGB")
        label = self.targets[index]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

def build_catdog_resnet_sets(folder="dogs-vs-cats/train"):

    dataset_pool = CatsDogsResized(
        folder,
        transform=resnet_pipeline
    )

    split_point = int(0.8 * len(dataset_pool))

    train_subset, val_subset = random_split(
        dataset_pool,
        [split_point, len(dataset_pool) - split_point]
    )

    return train_subset, val_subset


In [ ]:
import torch
import torch.nn as nn


class CompactCNN(nn.Module):

    def __init__(self,
                 classes=10,
                 activation_type="relu",
                 input_dims=(3, 32, 32)):

        super().__init__()

        self.activation = self._resolve_activation(activation_type)

        in_channels = input_dims[0]

        # ----- convolution stack -----
        self.stage1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32)
        )

        self.stage2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )

        self.stage3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128)
        )

        self.down = nn.MaxPool2d(2, 2)

        flat_dim = 128 * (input_dims[1] // 8) * (input_dims[2] // 8)

        # ----- classifier head -----
        self.hidden = nn.Linear(flat_dim, 256)
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(256, classes)

    # -------------------------------------------------

    def _resolve_activation(self, name):

        table = {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "leaky_relu": nn.LeakyReLU()
        }

        key = name.lower()

        if key not in table:
            raise ValueError("Unsupported activation")

        return table[key]

    # -------------------------------------------------

    def _feature_pipeline(self, x):

        for block in (self.stage1,
                      self.stage2,
                      self.stage3):

            x = self.down(self.activation(block(x)))

        return x

    # -------------------------------------------------

    def forward(self, x):

        x = self._feature_pipeline(x)

        x = torch.flatten(x, 1)

        x = self.hidden(x)
        x = self.activation(x)
        x = self.dropout(x)

        return self.classifier(x)


In [ ]:
import torch
import torch.nn as nn
import os
from torchvision import models

def create_resnet18(class_count, image_dims=(224, 224)):

    net = models.resnet18(weights=None)

    # ensure RGB input channel compatibility
    if net.conv1.in_channels != 3:
        net.conv1 = nn.Conv2d(
            3,
            64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )

    feature_dim = net.fc.in_features
    net.fc = nn.Linear(feature_dim, class_count)

    return net

def run_resnet_training(model,
                        dataset_tag,
                        train_dl,
                        val_dl,
                        optimizer,
                        loss_fn,
                        epochs=10,
                        device="cpu"):

    model.to(device)

    top_accuracy = 0.0

    for epoch_idx in range(epochs):

        # ---------- training phase ----------
        model.train()
        total_train_loss = 0.0

        for batch_x, batch_y in train_dl:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            predictions = model(batch_x)
            loss_value = loss_fn(predictions, batch_y)

            loss_value.backward()
            optimizer.step()

            total_train_loss += loss_value.item() * batch_x.size(0)

        train_loss = total_train_loss / len(train_dl.dataset)

        # ---------- validation phase ----------
        model.eval()

        correct_preds = 0
        sample_count = 0
        total_val_loss = 0.0

        with torch.no_grad():

            for batch_x, batch_y in val_dl:

                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)

                logits = model(batch_x)

                loss_value = loss_fn(logits, batch_y)
                total_val_loss += loss_value.item() * batch_x.size(0)

                _, predicted = torch.max(logits, dim=1)

                sample_count += batch_y.size(0)
                correct_preds += (predicted == batch_y).sum().item()

        val_loss = total_val_loss / len(val_dl.dataset)
        accuracy = 100.0 * correct_preds / sample_count

        print(
            f"Epoch {epoch_idx + 1}/{epochs} | "
            f"Train Loss: {train_loss:.4f} | "
            f"Val Loss: {val_loss:.4f} | "
            f"Val Acc: {accuracy:.2f}%"
        )

        # ---------- checkpointing ----------
        if accuracy > top_accuracy:

            top_accuracy = accuracy

            save_dir = os.path.join(
                "resnet_models",
                dataset_tag
            )

            os.makedirs(save_dir, exist_ok=True)

            torch.save(
                model.state_dict(),
                os.path.join(save_dir, "resnet18_best.pth")
            )

    print("Training finished — Best validation accuracy:", top_accuracy)


In [ ]:
def run_validation(model,
                   data_loader,
                   loss_fn,
                   device="cpu"):

    model.to(device)
    model.eval()

    correct_preds = 0
    sample_total = 0
    loss_total = 0.0

    with torch.no_grad():

        for batch_x, batch_y in data_loader:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            logits = model(batch_x)

            loss_value = loss_fn(logits, batch_y)
            loss_total += loss_value.item() * batch_x.size(0)

            _, predictions = torch.max(logits, dim=1)

            sample_total += batch_y.size(0)
            correct_preds += (predictions == batch_y).sum().item()

    accuracy = 100.0 * correct_preds / sample_total
    mean_loss = loss_total / sample_total

    return accuracy, mean_loss


In [ ]:
if torch.cuda.is_available():
    runtime_device = "cuda"
elif torch.backends.mps.is_available():
    runtime_device = "mps"
else:
    runtime_device = "cpu"

cifar_train_set, cifar_val_set, cifar_test_set = build_cifar_resnet_sets()

cifar_train_dl = DataLoader(
    cifar_train_set,
    batch_size=64,
    shuffle=True
)

cifar_val_dl = DataLoader(
    cifar_val_set,
    batch_size=64,
    shuffle=False
)

cifar_model = create_resnet18(class_count=10)

cifar_optimizer = optim.Adam(
    cifar_model.parameters(),
    lr=1e-4
)

loss_object = nn.CrossEntropyLoss()

run_resnet_training(
    cifar_model,
    dataset_tag="cifar",
    train_dl=cifar_train_dl,
    val_dl=cifar_val_dl,
    optimizer=cifar_optimizer,
    loss_fn=loss_object,
    epochs=10,
    device=runtime_device
)

dvc_train_set, dvc_val_set = build_catdog_resnet_sets()

dvc_train_dl = DataLoader(
    dvc_train_set,
    batch_size=32,
    shuffle=True
)

dvc_val_dl = DataLoader(
    dvc_val_set,
    batch_size=32,
    shuffle=False
)

dvc_model = create_resnet18(class_count=2)

dvc_optimizer = optim.Adam(
    dvc_model.parameters(),
    lr=1e-4
)

loss_object = nn.CrossEntropyLoss()

run_resnet_training(
    dvc_model,
    dataset_tag="dvc",
    train_dl=dvc_train_dl,
    val_dl=dvc_val_dl,
    optimizer=dvc_optimizer,
    loss_fn=loss_object,
    epochs=10,
    device=runtime_device
)


/var/folders/mz/34p9h3ld08ldtfp6qbl62tmc0000gn/T/ipykernel_45658/1812573342.py:9: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  batch = pickle.load(f, encoding='bytes')
/var/folders/mz/34p9h3ld08ldtfp6qbl62tmc0000gn/T/ipykernel_45658/1812573342.py:15: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  batch = pickle.load(f, encoding='bytes')


Epoch 1/10, Train Loss: 1.3414, Val Loss: 1.1672, Val Acc: 59.29%
Epoch 2/10, Train Loss: 0.9058, Val Loss: 0.9468, Val Acc: 66.90%
Epoch 3/10, Train Loss: 0.6853, Val Loss: 0.8507, Val Acc: 71.48%
Epoch 4/10, Train Loss: 0.5081, Val Loss: 0.7650, Val Acc: 73.51%
Epoch 5/10, Train Loss: 0.3389, Val Loss: 0.9067, Val Acc: 71.15%
Epoch 6/10, Train Loss: 0.2014, Val Loss: 0.9584, Val Acc: 72.77%
Epoch 7/10, Train Loss: 0.1146, Val Loss: 0.9036, Val Acc: 74.18%
Epoch 8/10, Train Loss: 0.0775, Val Loss: 0.9485, Val Acc: 73.82%
Epoch 9/10, Train Loss: 0.0701, Val Loss: 1.0792, Val Acc: 72.28%
Epoch 10/10, Train Loss: 0.0618, Val Loss: 1.0741, Val Acc: 73.50%
Training complete. Best Val Accuracy: 74.18
Epoch 1/10, Train Loss: 0.5482, Val Loss: 0.4396, Val Acc: 79.44%
Epoch 2/10, Train Loss: 0.3881, Val Loss: 0.3817, Val Acc: 82.04%
Epoch 3/10, Train Loss: 0.2794, Val Loss: 0.2527, Val Acc: 89.32%
Epoch 4/10, Train Loss: 0.1943, Val Loss: 0.2450, Val Acc: 89.22%
Epoch 5/10, Train Loss: 0.1251,

In [ ]:
results_table = pd.read_csv("experiment_results.csv")

top_configs = {
    "cifar": results_table
        .query("dataset == 'Cifar-10'")
        .sort_values("accuracy", ascending=False)
        .head(1),

    "dvc": results_table
        .query("dataset == 'Dogs vs Cats'")
        .sort_values("accuracy", ascending=False)
        .head(1)
}

def compose_checkpoint_path(dataset_key, row):

    return (
        f"models/{dataset_key}/model_"
        f"{row['activation']}_"
        f"{row['init']}_"
        f"{row['optimizer']}_best.pth"
    )


cifar_best_path = compose_checkpoint_path(
    "cifar",
    top_configs["cifar"].iloc[0]
)

dvc_best_path = compose_checkpoint_path(
    "dvc",
    top_configs["dvc"].iloc[0]
)

print("Best CIFAR-10 model path:", cifar_best_path)
print("Best Dogs vs Cats model path:", dvc_best_path)


Best CIFAR-10 model path: models/cifar/model_leaky_relu_random_rmsprop_best.pth
Best Dogs vs Cats model path: models/dvc/model_leaky_relu_kaiming_adam_best.pth


In [ ]:

# LOSS FUNCTION + RESULT STORAGE

loss_fn = nn.CrossEntropyLoss()
comparison_rows = []

cifar_resnet = create_resnet18(class_count=10)
cifar_resnet.load_state_dict(
    torch.load(
        "resnet_models/cifar/resnet18_best.pth",
        map_location=runtime_device
    )
)

cifar_resnet_acc, cifar_resnet_loss = run_validation(
    cifar_resnet,
    cifar_val_dl,
    loss_fn,
    runtime_device
)

comparison_rows.append({
    "dataset": "CIFAR-10",
    "model": "ResNet-18",
    "accuracy": cifar_resnet_acc,
    "loss": cifar_resnet_loss
})


# CIFAR — BEST CUSTOM MODEL (from experiment table)

cifar_best_cfg = top_configs["cifar"].iloc[0]

comparison_rows.append({
    "dataset": "CIFAR-10",
    "model": (
        f"{cifar_best_cfg['activation']}_"
        f"{cifar_best_cfg['init']}_"
        f"{cifar_best_cfg['optimizer']}"
    ),
    "accuracy": cifar_best_cfg["accuracy"],
    "loss": cifar_best_cfg["val_loss"]
})

dvc_resnet = create_resnet18(class_count=2)
dvc_resnet.load_state_dict(
    torch.load(
        "resnet_models/dvc/resnet18_best.pth",
        map_location=runtime_device
    )
)

dvc_resnet_acc, dvc_resnet_loss = run_validation(
    dvc_resnet,
    dvc_val_dl,
    loss_fn,
    runtime_device
)

comparison_rows.append({
    "dataset": "Dogs vs Cats",
    "model": "ResNet-18",
    "accuracy": dvc_resnet_acc,
    "loss": dvc_resnet_loss
})


# DOGS vs CATS — BEST CUSTOM MODEL

dvc_best_cfg = top_configs["dvc"].iloc[0]

comparison_rows.append({
    "dataset": "Dogs vs Cats",
    "model": (
        f"{dvc_best_cfg['activation']}_"
        f"{dvc_best_cfg['init']}_"
        f"{dvc_best_cfg['optimizer']}"
    ),
    "accuracy": dvc_best_cfg["accuracy"],
    "loss": dvc_best_cfg["val_loss"]
})

comparison_df = pd.DataFrame(comparison_rows)
comparison_df.to_csv("comparison_results.csv", index=False)

print("Comparison saved to comparison_results.csv")


Comparison saved to comparison_results.csv


In [ ]:
df = pd.DataFrame(pd.read_csv('comparison_results.csv'))
df

,dataset,model,accuracy,loss
0,CIFAR-10,ResNet-18,74.18,0.903581
1,CIFAR-10,leaky_relu_random_rmsprop,84.03,0.463258
2,Dogs vs Cats,ResNet-18,90.22,0.255094
3,Dogs vs Cats,leaky_relu_kaiming_adam,89.74,0.234952
